<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/mlp_imdb_hf_dset_and_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Before we start running our own Python code, install the required Python packages using [pip](https://en.wikipedia.org/wiki/Pip):

* [`transformers`](https://huggingface.co/docs/transformers/index) is a popular deep learning package primarily on top of torch, we need to reinstall it with the [torch] configuration (might take a substantial amount of time)
* [`datasets`](https://huggingface.co/docs/datasets/) provides support for loading, creating, and manipulating datasets
* evaluate is a library of performance metrics (like accuracy etc)

**You will likely need to do a Runtime/Restart session for everything to work after the installation.**

In [13]:
# !pip3 install -q datasets evaluate
# !pip install transformers[torch]

(Above, the `!` at the start of the line tells the notebook to run the line as an operating system command rather than Python code, and the `-q` argument to `pip` runs the command in "quiet" mode, with less output.)

---

# Get and prepare data

*   Let us work with the IMDB dataset of movie review sentiment
*   25,000 positive reviews
*   25,000 negative reviews
*   50,000 unlabeled reviews (which we discard for the time being)


In [14]:
from pprint import pprint #pprint => pretty-print, I use it occassionally throughout the notebook
import datasets
import torch
dset=datasets.load_dataset("imdb")
pprint(dset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [15]:
dset=dset.shuffle() #This is never a bad idea, datasets may have ordering to them, which is not what we want
del dset["unsupervised"] #Delete the unlabeled part of the dataset, we don't need it for anything

In [16]:
pprint(dset['train'][0]['text'])
print(dset['train'][0]['label'])

('I happened on "Shower" in the foreign film section of my local video store '
 'and passed it over several times since from its cover it looked like a farce '
 'or comedy. I then lucked into a copy to purchase at economical price and am '
 'happy for my luck. "Shower" is the story of three(3) men, a father and '
 'two(2) adult sons, each coming to terms with life changes as the world '
 'around them also continues to change in modern China. As with many "foreign" '
 'films, the Chinese culture itself is one of the most interesting facets of '
 'this movie.<br /><br />Beyond the fascinating characteristics of the local, '
 'Chinese color giving the setting to this story, is the difficult yet '
 'touching relationships between the men and a sole woman involved in the '
 'story, all set against the backdrop of a village bathhouse.<br /><br />The '
 "family's story moves from estrangement to understanding and made me glad I "
 'came to know these people. Added to the main story are the nu

## Tokenize and map vocabulary
         
*   We need to achieve two complementary tasks
*   **Tokenize** split the text into units which can be interpreted as features (words in this case)
*   **Map vocabulary** build the feature vector for each example
*   Since this is NLP, here it means listing the non-zero elements of the feature vector, or in other words the indices of the vocabulary items
* Since we work with the bag of words (BoW) representation, these do not need to be (and are not) in the order in which they appear in the text
* These indices then refer to the rows in the embedding matrix
*   A traditional and well-tested way it to use sklearn's feature extraction package
*   CountVectorizer is most likely what we want in here, because we only want the ids, nothing else
* But for other NLP work the TfidfVectorizer is also very handy



In [17]:
import sklearn.feature_extraction

# max_features means the size of the vocabulary
# which means max_features most-common words
vectorizer=sklearn.feature_extraction.text.CountVectorizer(binary=True,max_features=20000)

texts=[ex["text"] for ex in dset["train"]] #get a list of all texts from the training data
vectorizer.fit(texts) #"Trains" the vectorizer, i.e. builds its vocabulary


CountVectorizer(binary=True, max_features=20000)

# Building the feature vectors

* This is super-easy with the vectorizer
* It produces a sparse matrix of the non-zero elements

In [18]:
def vectorize_example(ex):
    vectorized=vectorizer.transform([ex["text"]]) # [...] because the vectorizer expects a list/iterable over inputs, not one input
    non_zero_features=vectorized.nonzero()[1] #.nonzero gives a pair of (rows,columns), we want the columns
    non_zero_features+=1 #feature index 0 will have a special meaning
                         # so let us not produce it by adding +1 to everything
    return {"input_ids":non_zero_features}

vectorized=vectorize_example(dset["train"][0])

In [19]:
print(vectorized)

{'input_ids': array([  451,   521,   606,   727,   774,   794,   887,  1115,  1157,
        1207,  1300,  1456,  1497,  1901,  1911,  2295,  2625,  2687,
        3037,  3040,  3058,  3062,  3187,  3189,  3560,  3599,  3616,
        3842,  3979,  4071,  4172,  4424,  4474,  5108,  5736,  5791,
        6578,  6640,  6664,  6683,  6706,  6878,  6887,  7127,  7147,
        7331,  7346,  7695,  7697,  8225,  8233,  8811,  9085,  9180,
        9428,  9485,  9557,  9602,  9630,  9638,  9639, 10090, 10449,
       10475, 10592, 10640, 10734, 10829, 10871, 10970, 11176, 11267,
       11576, 11721, 11761, 11762, 11857, 12260, 12363, 12437, 12439,
       12504, 12627, 12887, 13028, 13727, 14034, 14574, 15667, 15812,
       15816, 15830, 16041, 16161, 16381, 16404, 16522, 16565, 17068,
       17075, 17837, 17897, 17907, 17910, 17917, 17938, 17968, 18000,
       18081, 18115, 18217, 18219, 18553, 18685, 19144, 19173, 19325,
       19377, 19712, 19741, 19785, 19914])}


In [20]:
# We can map back to vocabulary and check that everything works
# vectorizer.vocabulary_ is a dictionary {key:word, value:idx}

idx2word=dict((i,w) for (w,i) in vectorizer.vocabulary_.items()) #inverse the vocab dictionary
words=[]
for idx in vectorized["input_ids"]:
    words.append(idx2word[idx-1]) ## It is easy to forgot we moved all by +1
pprint(", ".join(words)) #This is now the bag of words representation of the document

('added, adult, against, all, also, am, and, are, around, as, at, away, '
 'backdrop, between, beyond, br, by, came, change, changes, characteristics, '
 'characters, china, chinese, color, comedy, coming, conflicts, continues, '
 'copy, cover, culture, customers, difficult, each, economical, facets, '
 'family, farce, fascinating, father, film, films, for, foreign, friendships, '
 'from, giving, glad, happened, happy, humanity, in, individual, interesting, '
 'into, involved, is, it, its, itself, know, life, like, local, looked, luck, '
 'made, main, many, me, men, modern, most, moves, movie, my, numerous, of, on, '
 'one, or, over, passed, people, price, purchase, relationships, section, set, '
 'setting, several, shower, since, small, smiling, sole, sons, store, story, '
 'terms, the, their, them, then, these, this, three, times, to, touched, '
 'touching, two, understanding, video, village, walks, warmth, with, woman, '
 'world, yet')


# Tokenizing / vectorizing the whole dataset

* The datasets library allows us to efficiently map() a function across the whole dataset
* Can run in parallel

**Note**: confusingly, and unlike the Python`map` function, [`Dataset.map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) function _updates_ its argument dataset, keeping existing values. Here, the call adds the values returned by the function call (here `input_ids`) to each example while also keeping the original `text` and `label` values.


In [21]:
# Apply the tokenizer to the whole dataset using .map()
dset_tokenized = dset.map(vectorize_example)
pprint(dset_tokenized["train"][0])

Map: 100%|██████████| 25000/25000 [00:10<00:00, 2287.36 examples/s]

{'input_ids': [451,
               521,
               606,
               727,
               774,
               794,
               887,
               1115,
               1157,
               1207,
               1300,
               1456,
               1497,
               1901,
               1911,
               2295,
               2625,
               2687,
               3037,
               3040,
               3058,
               3062,
               3187,
               3189,
               3560,
               3599,
               3616,
               3842,
               3979,
               4071,
               4172,
               4424,
               4474,
               5108,
               5736,
               5791,
               6578,
               6640,
               6664,
               6683,
               6706,
               6878,
               6887,
               7127,
               7147,
               7331,
               7346,
               7695,

## Input encoding for MLP

* Our `input_ids` are an array containing the indices of the tokens found in the text
* This corresponds to the indices into the row of the embedding matrix in the model
* That seems to be exactly what we need!


# Batching and padding

* When working with neural networks, one rarely trains one example at a time
* Instead, processing always happens a batch at a time
* This has two important reasons:
  1. No batching is too slow (GPU parallelization cannot kick in across examples)
  2. The gradients are averaged across the whole batch and applied only once, i.e. batching acts as a regularizer and improves the stability of the training


# Padding and Collation (forming a batch)

## Padding:

* In order to build a batch as a 2D array of (example, seq), we need to fit together examples of different length
* Solution: pad the shorter examples with zeroes to the length of the longest example in the batch
* Make sure that zero is understood as padding value rather than a (hypothetical) feature with index 0
* This is best shown by example, it is in the end easier than it may sound

## Collation:

* Much like examples are dictionaries with the data, also batches are dictionaries with the data
* The only difference is that in a batch, all data tensors have one extra dimension, that's all there is to it

## Collator function:

* Padding and collation is taken care of by a single function in the HF libraries
* It receives a list of examples, and returns a ready batch
* The surrounding library code takes care of forming these lists
* Let's try to implement one below

In [22]:
# 1) I need to define it here, will explain below
# 2) I show here a very straightforward implementation of padding and collation
# 3) Normally, one would use transformers.DataCollatorWithPadding but that assumes
#    a particular tokenizer, to which it outsources much of the work, and we do not
#    have it
def collator(list_of_examples):
    #this is easy, labels are made into a single tensor
    batch={"labels":torch.tensor(list(ex["label"] for ex in list_of_examples))}
    #the worse bit is now to pad the examples, as they are of different length
    tensors=[]
    max_len=max(len(example["input_ids"]) for example in list_of_examples) #this is the longest example in the batch
    #everything needs to be padded to fit in length the longest example
    #(so we can build a single tensor out of it)
    for example in list_of_examples:
        ids=torch.tensor(example["input_ids"]) #pick the input ids
        # pad(what,(from_left, from_right)) <- this is how we call the stock pad function
        padded=torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])) #pad by max - current length, pads with zero by default
        tensors.append(padded) #accumulated the padded ids
    batch["input_ids"]=torch.vstack(tensors) #now that we have all of them the same length, a simple vstack() stacks them up
    return batch #...and that's all there is to it



#Build a batch from 2 examples, with padding
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
pprint(batch["labels"])
pprint(batch["input_ids"])

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 99])
tensor([0, 0])
tensor([[   10,   309,   887,   954,  1115,  1237,  1517,  1573,  1679,  1702,
          2454,  2604,  2625,  2687,  2711,  2717,  2958,  3637,  4130,  4225,
          4723,  5091,  5429,  6302,  6319,  6642,  6767,  6768,  7627,  7692,
          7932,  8047,  8242,  8303,  8322,  8350,  8379,  8780,  9302,  9377,
          9602,  9630,  9890, 10008, 10475, 10638, 10798, 10829, 11176, 11681,
         11857, 11897, 11936, 12134, 12142, 12256, 12363, 12437, 12439, 12577,
         12839, 13323, 13778, 14323, 14349, 15339, 15461, 15464, 16020, 16033,
         16478, 16568, 17302, 17309, 17636, 17649, 17885, 17893, 17897, 17907,
         17910, 17916, 17929, 17944, 17968, 18069, 18115, 18166, 18922, 19332,
         19355, 19521, 19540, 19639, 19712, 19805, 19813, 19832, 19843],
        [  309,   540,   720,   727,   764,   887,  1007,  1115,  1157,  1517,
          1523,  1744,  2516,  2784,  2924,  3199

# Build the MLP model

* Now that all of our data is in shape, we can build the model
* That is luckily quite easy in this case

The model class in its simplest form has `__init__()` which instantiates the layers and `forward()` which implements the actual computation. For more information on these, please see the [PyTorch turorial](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html).

In [23]:
import torch
import transformers

# A model wants a config, I can simply inherit from the base
# class for pretrained configs
class MLPConfig(transformers.PretrainedConfig):
    pass

# This is the model
class MLP(transformers.PreTrainedModel):

    config_class=MLPConfig

    # In the initialization method, one instantiates the layers
    # these will be, for the most part the trained parameters of the model
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        # Build and initialize embedding of vocab size +1 x hidden size (+1 because of the padding index 0!)
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        # Normally you would not initialize these yourself, but I have my reasons here ;)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) #initialize the embeddings with small random values
        # Note! This function is relatively clever and keeps the embedding for 0, the padding, pure zeros
        # This takes care of the lower half of the network, now the upper half
        # Output layer: hidden size x output size
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)
        # Now we have the parameters of the model


    # The computation of the model is put into the forward() function
    # it receives a batch of data and optionally the correct `labels`
    #
    # If given `labels` it returns (loss,output)
    # if not, then it returns (output,)
    def forward(self,input_ids,labels=None):
        #1) sum up the embeddings of the items
        embedded=self.embedding(input_ids) #(batch,ids)->(batch,ids,embedding_dim)
        # Since the Embedding keeps the first row of the matrix pure zeros, we don't need to worry about the padding
        # so next we sum the embeddings across the word dimension
        # (batch,ids,embedding_dim) -> (batch,embedding_dim)
        embedded_summed=torch.sum(embedded,dim=1)

        #2) apply non-linearity
        # (batch,embedding_dim) -> (batch,embedding_dim)
        projected=torch.tanh(embedded_summed) #Note how non-linearity is applied here and not when configuring the layer in __init__()

        #3) and now apply the upper, output layer of the network
        # (batch,embedding_dim) -> (batch, num_of_classes i.e. 2 in our case)
        logits=self.output(projected)

        # ...and that's all there is to it!

        #print("input_ids.shape",input_ids.shape)
        #print("embedded.shape",embedded.shape)
        #print("embedded_summed.shape",embedded_summed.shape)
        #print("projected.shape",projected.shape)
        #print("logits.shape",logits.shape)

        # If we have labels, we ought to calculate the loss
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss() #This loss is meant for classification, so let's use it
            # You run it as loss(model_output,correct_labels)
            return (loss(logits,labels),logits)
        else:
            # No labels, so just return the logits
            return (logits,)



In [24]:
# Configure the model:
#   these parameters are used in the model's __init__()
mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=1,nlabels=2)

# And now we can instantiate it
mlp=MLP(mlp_config)

#we can make a little test with a fake batch formed by the two first example
fake_batch=collator([dset_tokenized["train"][0],dset_tokenized["train"][1]])
mlp(**fake_batch) #** expands input_ids and labels as parameters of the call

(tensor(0.7724, grad_fn=<NllLossBackward0>),
 tensor([[-0.4547,  0.3354],
         [-0.4613,  0.3380]], grad_fn=<AddmmBackward0>))

# Train the model

We will use the Hugging Face [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) class for training

* Loads of arguments that control the training
* Configurable metrics to evaluate performance
* Data collator builds the batches
* Early stopping callback stops when eval loss no longer improves
* Model load/save
* Excellent foundation for later deep learning course
  

First, let's create a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/trainer#transformers.TrainingArguments) object to specify hyperparameters and various other settings for training.

Printing this simple dataclass object will show not only the values we set, but also the defaults for all other arguments. Don't worry if you don't understand what all of these do! Many are not relevant to us here, and you can find the details in [`Trainer` documentation](https://huggingface.co/docs/transformers/main_classes/trainer) if you are interested.

In [25]:
# Set training arguments
# their names are mostly self-explanatory
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-5, #learning rate of the gradient descent
    max_steps=20000,
    load_best_model_at_end=True,
    per_device_train_batch_size=128
)

pprint(trainer_args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

Next, let's create a metric for evaluating performance during and after training. We can use the convenience function [`load_metric`](https://huggingface.co/docs/datasets/about_metrics) to load one of many pre-made metrics and wrap this for use by the trainer.

As the task is simple binary classification and our data is even 50:50 balanced, we can comfortably use the basic `accuracy` metric, defined as the proportion of correctly predicted labels out of all labels.

In [26]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

We can then create the `Trainer` and train the model by invoking the [`Trainer.train`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.train) function.

In addition to the model, the settings passed in through the `TrainingArguments` object created above (`trainer_args`), the data, and the metric defined above, we create and pass the following to the `Trainer`:

* [data collator](https://huggingface.co/docs/transformers/main_classes/data_collator): groups input into batches
* [`EarlyStoppingCallback`](https://huggingface.co/docs/transformers/main_classes/callback#transformers.EarlyStoppingCallback): stops training when performance stops improving

In [27]:
# Make a new model
mlp = MLP(mlp_config)


# Argument gives the number of steps of patience before early stopping
# i.e. training is stopped when the evaluation loss fails to improve
# certain number of times
early_stopping = transformers.EarlyStoppingCallback(5)

trainer = transformers.Trainer(
    model=mlp,
    args=trainer_args,
    train_dataset=dset_tokenized["train"],
    eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

# FINALLY!
trainer.train()

c:\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  2%|▎         | 500/20000 [00:11<06:41, 48.60it/s]

{'loss': 0.7903, 'grad_norm': 0.7438209056854248, 'learning_rate': 9.75e-06, 'epoch': 2.55}


                                                   
  3%|▎         | 507/20000 [00:11<10:36, 30.64it/s]

{'eval_loss': 0.7590334415435791, 'eval_accuracy': 0.51, 'eval_runtime': 0.2868, 'eval_samples_per_second': 3487.041, 'eval_steps_per_second': 435.88, 'epoch': 2.55}


  5%|▌         | 1000/20000 [00:22<06:26, 49.13it/s]

...

100%|██████████| 20000/20000 [07:45<00:00, 46.67it/s]

{'loss': 0.5487, 'grad_norm': 0.4650239944458008, 'learning_rate': 0.0, 'epoch': 102.04}


                                                     
100%|██████████| 20000/20000 [07:45<00:00, 42.98it/s]

{'eval_loss': 0.5595652461051941, 'eval_accuracy': 0.659, 'eval_runtime': 0.2758, 'eval_samples_per_second': 3625.8, 'eval_steps_per_second': 453.225, 'epoch': 102.04}
{'train_runtime': 465.3703, 'train_samples_per_second': 5500.995, 'train_steps_per_second': 42.977, 'train_loss': 0.6097665313720703, 'epoch': 102.04}


TrainOutput(global_step=20000, training_loss=0.6097665313720703, metrics={'train_runtime': 465.3703, 'train_samples_per_second': 5500.995, 'train_steps_per_second': 42.977, 'train_loss': 0.6097665313720703, 'epoch': 102.04})

We can then evaluate the trained model on a given dataset (here our test subset) by calling [`Trainer.evaluate`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.evaluate):

In [28]:
eval_results = trainer.evaluate(dset_tokenized["test"])

print(eval_results)

100%|██████████| 3125/3125 [00:06<00:00, 510.21it/s]

{'eval_loss': 0.5708923935890198, 'eval_accuracy': 0.6408, 'eval_runtime': 6.1279, 'eval_samples_per_second': 4079.695, 'eval_steps_per_second': 509.962, 'epoch': 102.04}


# Save the model for later use

* You can save it with `trainer.save_model()`
* You can load it with `MLP.from_pretrained()`


In [29]:
trainer.save_model("mlp-imdb")

# Check save/load

In [30]:
mlp2=MLP.from_pretrained("mlp-imdb")

In [31]:
trainer = transformers.Trainer(
    model=mlp2,
    args=trainer_args,
    train_dataset=dset_tokenized["train"],
    eval_dataset=dset_tokenized["test"],
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

c:\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [32]:
eval_results = trainer.evaluate(dset_tokenized["test"])
print(eval_results)
print('Accuracy:', eval_results['eval_accuracy'])

  2%|▏         | 57/3125 [00:00<00:05, 548.49it/s]

100%|██████████| 3125/3125 [00:05<00:00, 526.74it/s]

{'eval_loss': 0.5708923935890198, 'eval_accuracy': 0.6408, 'eval_runtime': 5.9388, 'eval_samples_per_second': 4209.634, 'eval_steps_per_second': 526.204}
Accuracy: 0.6408


# Extra time left?

* Read through the TrainingArguments documentation, try to understand at least some parts of it https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
* Read through Torch tensor operations, try to understand at least some parts of it: https://pytorch.org/docs/stable/tensors.html
* Run the model with different parameters (hidden layer width, learning rate, etc), how much do the results change?


# What has the model learned?

* The embeddings should have some meaning to them
* Similar features should have similar embeddings

In [33]:
# Grab the embedding matrix out of the trained model
# and drop the first row (padding 0)
# then we can treat the embeddings as vectors
# and maybe compare them to each other
# ha ha this below took some googling
weights=mlp.embedding.weight.detach().cpu().numpy()
weights=weights[1:,:]

In [34]:
qry_idx=vectorizer.vocabulary_["lousy"] #embedding of "great"

#calculate the distance of the "lousy" embedding to all other embeddings
distance_to_qry=sklearn.metrics.pairwise.euclidean_distances(weights[qry_idx:qry_idx+1,:],weights)
nearest_neighbors=np.argsort(distance_to_qry) #indices of words nearest to "lousy"
for nearest in nearest_neighbors[0,:20]:
    print(idx2word[nearest])
# This works great!

lousy
atrocious
excuse
bored
unbelievable
joke
flat
bother
insult
sucks
decent
uninteresting
weak
dreadful
acting
incoherent
amateurish
low
couldn
sorry


In [35]:
print(nearest_neighbors)

[[10693  1315  6398 ...  7931 19747  6366]]


* The embeddings indeed seem to reflect the task
* There is a meaning to them

# Feature weights

*   A typical "old-school" way to approach the classification would be a simple linear model, like LinearSVM
*   Under such model, each feature (word) would have a single one weight
*   And the classification would simply be based on the sum of these weights
*   In this context of this task, "positive" words would get a high weight, "negative" words would get a low weight
*   It is in fact quite easy to reconfigure the MLP model to work more or less like this and this effect can be replicated
*   I will leave that as an exercise for you



In [36]:
# weights now looks like this
print(weights)

[[ 0.00931336]
 [ 0.00958689]
 [-0.0108391 ]
 ...
 [-0.01083061]
 [ 0.01148495]
 [ 0.00241483]]


In [37]:
# we dont want each weight to be an array item so lets reshape
weights = weights.reshape(1, -1)

In [38]:
# print the hundred most positive words
sorted_feature_importances=np.argsort(weights)
for most_positive in sorted_feature_importances[0,:100]:
    print(idx2word[most_positive])

excellent
wonderful
great
perfect
amazing
favorite
superb
loved
fantastic
highly
best
beautiful
brilliant
wonderfully
touching
today
love
beautifully
gem
enjoyed
heart
terrific
perfectly
refreshing
recommended
powerful
incredible
captures
rare
outstanding
unique
always
underrated
performances
delightful
job
subtle
definitely
both
moving
finest
also
simple
fun
strong
flawless
emotions
awesome
true
noir
superbly
favorites
masterpiece
brilliantly
realistic
sweet
enjoyable
well
enjoy
different
favourite
greatest
friendship
perfection
breathtaking
stunning
solid
classic
journey
shows
magnificent
gives
especially
beauty
delight
excellently
performance
each
world
timeless
intense
portrayal
funniest
complex
still
atmosphere
tears
haunting
fascinating
appreciated
extraordinary
family
notch
helps
remarkable
unexpected
deeply
chilling
freedom
years


In [56]:
# print the hundred most negative words
for most_negative in reversed(sorted_feature_importances[0,-100:]):
    print(idx2word[most_negative])

worst
bad
waste
awful
terrible
boring
worse
stupid
horrible
poor
poorly
nothing
dull
crap
lame
avoid
pointless
ridiculous
wasted
supposed
mess
minutes
laughable
badly
pathetic
no
annoying
script
fails
money
plot
unfortunately
redeeming
disappointing
save
unfunny
unless
cheap
instead
oh
disappointment
predictable
garbage
attempt
dumb
wooden
mediocre
sorry
acting
decent
joke
atrocious
lousy
excuse
bored
unbelievable
flat
bother
insult
sucks
uninteresting
weak
dreadful
incoherent
amateurish
low
couldn
reason
lacks
remotely
tedious
why
even
silly
failed
idea
mst3k
guess
none
embarrassing
half
ok
whatsoever
rubbish
effort
wasting
least
unwatchable
costs
better
skip
forgettable
fake
unconvincing
trash
any
bunch
bland
pretentious
don
